In [1]:
%reload_ext autoreload
%autoreload 2

In [3]:
# !pip install peft bitsandbytes -q
from matplotlib import pyplot as plt
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
from open_pref_eval.datasets import get_default_datasets
from open_pref_eval.evaluation import evaluate_models, evaluate_model, evaluate
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import torch
from transformers import BitsAndBytesConfig
from tqdm.auto import tqdm
from anycache import anycache
from open_pref_eval.plot.radar import radar_plot
from open_pref_eval.helpers.mem import clear_mem


quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
)

In [4]:
datasets = get_default_datasets(300)
datasets

[Dataset({
     features: ['prompt', 'chosen', 'rejected', 'id'],
     num_rows: 300
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected'],
     num_rows: 300
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected'],
     num_rows: 300
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected'],
     num_rows: 300
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected'],
     num_rows: 300
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected'],
     num_rows: 300
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected'],
     num_rows: 300
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected'],
     num_rows: 300
 })]

In [5]:
models = [
    'snake7gun/tiny-random-qwen3',

    "unsloth/Qwen3-4B",
    "fakezeta/amoral-Qwen3-4B",
    "soob3123/amoral-qwen3-14B",
    
    "mlabonne/Qwen3-4B-abliterated",

    "wassname/qwen-7B-codefourchan",
    # "unsloth/gemma-2-9b-it",
    "opencompass/CompassJudger-1-7B-Instruct",
    "microsoft/Phi-4-mini-instruct",
    # "CohereLabs/c4ai-command-r7b-12-2024",
    # 'emergent-misalignment/Qwen-Coder-Insecure',
    # "drfellx/emergent_misalignment_test_qwen2.5-7B-Instruct",
    # "dpasch01/pp-llama3-8b-right-wing",
    # "dpasch01/pp-llama3-8b-left-wing",
    # "unsloth/DeepSeek-R1-Distill-Qwen-7B"
]

In [6]:
from open_pref_eval.scoring import first_nonzero, score_1st_diverg, score_preferences, score_ipo, score_cumsum, score_weighted, score_weighted_prob, score_with_weight, score_f_alpha_divergance, score_f_divergance, score_agg, score_certainty_weighted, score_entropy_weighted, score_power_mean, score_with_decay, score_percentile, score_1st_diverg, score_perplexity_ratio
score_fns = {
    "preferences": score_preferences,
    "cumsum": score_cumsum,
    "weighted": score_weighted, # littlediff
    "weighted_prob": score_weighted_prob, # nan
    "with_weight": score_with_weight,
    "f_alpha_divergance": score_f_alpha_divergance,
    "f_divergance": score_f_divergance,
    "min": lambda *args, **kwargs: score_agg(*args, **kwargs, agg=lambda x: torch.min(x, dim=-1).values),
    "max": lambda *args, **kwargs: score_agg(*args, **kwargs, agg=lambda x: torch.max(x, dim=-1).values),
    "mean": lambda *args, **kwargs: score_agg(*args, **kwargs, agg=lambda x: torch.mean(x, dim=-1)),
    "median": lambda *args, **kwargs: score_agg(*args, **kwargs, agg=lambda x: torch.median(x, dim=-1).values),
    "std": lambda *args, **kwargs: score_agg(*args, **kwargs, agg=lambda x: torch.std(x, dim=-1)),
    "ipo": score_ipo,
    "entropy_weighted": score_entropy_weighted,
    "certainty_weighted": score_certainty_weighted,
    "power_mean": score_power_mean,
    "with_decay": score_with_decay,
    "percentile": score_percentile,
    "1st_diverg": score_1st_diverg,
    "perplexity_ratio": score_perplexity_ratio,

}


In [ ]:
@anycache('.anycache')
def eval_model(model_id):
    print(f"Evaluating {model_id}")
    tokenizer = AutoTokenizer.from_pretrained(model_id, ) #trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(model_id, 
        torch_dtype=torch.bfloat16, 
        quantization_config=quantization_config, device_map="auto", 
    # trust_remote_code=True
    )
    
    # Evaluate the model
    results, df_raw = evaluate_model(
        model=model,
        tokenizer=tokenizer,
        datasets=datasets,
        batch_size=6,
        max_length=1024,
        max_prompt_length=512,
        verbose=2,
        score_fn=score_fns
    ) 
    return results, df_raw

data = []
for model_id in tqdm(models):
    results, df_raw = eval_model(model_id)
    clear_mem()
    data.append(df_raw)

  0%|          | 0/8 [00:00<?, ?it/s]

Evaluating snake7gun/tiny-random-qwen3


  0%|          | 0/8 [00:00<?, ?dataset/s]

Eval toxic-dpo-v0.2-train[:300]:   0%|          | 0/50 [00:00<?, ?it/s]

You're using a Qwen2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/media/wassname/SGIronWolf/projects5/elk/open_pref_eval/.venv/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
2025-05-24 14:16:02.350 | DEBUG    | open_pref_eval.trainer:__call__:91 - Batch Chosen were truncated to 512 tokens for 16.67% of samples. Consider increasing max_completion_length.


...

model-00001-of-00006.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00006-of-00006.safetensors:   0%|          | 0.00/4.73G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00006.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
import pandas as pd
df = pd.concat(data)

df.groupby(['model', 'dataset'])['correct'].mean().unstack()

In [ ]:
def rename_ds(s):
    s = s.replace('_preferences', '')
    s = s.replace('ethics_', '')
    s = s.replace('mmlu-', '')
    s = s.replace('validation', 'test')
    s = s.replace('train', 'test')
    s = '-'.join(s.split('-test')[:-1])
    return s

df2 = df.copy()
df2['dataset'] = df2['dataset'].apply(rename_ds)
df2['dataset']

In [ ]:
df2.dataset.unique()

In [ ]:
import numpy as np


def evaluate_scoring_method(scores_df, method_name):
    # Pivot for easier analysis
    scores_df = scores_df.copy()
    scores_df[method_name] = scores_df[method_name].replace([np.inf, -np.inf], np.nan)
    scores = pd.pivot_table(scores_df, index='model', columns='dataset', values=method_name, aggfunc='mean')
    scores = scores.clip(lower=0, upper=1)  # Clip scores to [0, 1] range
    # print(scores)
    
    # 1. IMDB should be high (most models > 0.8)
    imdb_score = scores['imdb'].mean()
    
    # 2. Hard datasets should be low (if you have a hard_math dataset)
    hard_math_score = scores['elementary_mathematics'].mean()# if 'elementary_mathematics' in scores else 0.5
    hard_math_penalty = 1 - abs(hard_math_score - 0.5)

    # 3. Random model should be ~0.5
    random_model = 'snake7gun/tiny-random-qwen3'  # your random model
    random_deviation = abs(scores.loc[random_model].mean() - 0.5)
    random_penalty = 1 - random_deviation  # 1 is good, 0 is bad
    
    # 4. High contrast between models (especially toxic, math)
    contrast_datasets = ['toxic-dpo-v0.2', 'elementary_mathematics', 'expression-justice']
    contrasts = [scores[ds].std() for ds in contrast_datasets if ds in scores]
    avg_contrast = np.mean(contrasts)

    # 3. Discrimination - use robust measure
    contrast_datasets = ['toxic-dpo-v0.2', 'elementary_mathematics', 'expression-justice']
    contrasts = []
    for ds in contrast_datasets:
        if ds in scores and not scores[ds].isna().all():
            # Use IQR instead of std - more robust to outliers
            q75, q25 = scores[ds].quantile(0.75), scores[ds].quantile(0.25)
            iqr = q75 - q25
            # Normalize by median to make it scale-invariant
            median = scores[ds].median()
            if median > 0:
                contrasts.append(min(iqr / median, 2))  # cap at 2
    
    avg_contrast = np.mean(contrasts) if contrasts else 0
    
    # 5. Bonus: censored vs uncensored should differ on toxic
    if 'toxic-dpo-v0.2' in scores:
        # Assuming censored models score low, uncensored high
        toxic_spread = scores['toxic-dpo-v0.2'].max() - scores['toxic-dpo-v0.2'].min()
    else:
        toxic_spread = 0
    
    # Combined score
    quality = (
        imdb_score * 2 +              # weight easy dataset performance
        random_penalty * 3 +          # important: random = 0.5
        avg_contrast * 2 +            # discrimination power
        toxic_spread +                 # specific contrast we expect
        hard_math_penalty               # weight hard dataset performance
    ) / 10  # normalize to [0, 1]

    return {
        'overall': quality,
        'imdb_mean': imdb_score,
        'random_calibration': random_penalty,
        'discrimination': avg_contrast,
        'toxic_spread': toxic_spread,
        'hard_math': hard_math_score,

    }


cols = [c for c in df2.columns if 'score' in c]
res = {}
for c in cols:
    v = evaluate_scoring_method(df2, c)
    res[c] = v

    df2['correct2'] = df2[c]>0.5
    v = evaluate_scoring_method(df2, 'correct2')
    res[f'{c}_bool'] = v


res = pd.DataFrame(res).T.sort_values('overall', ascending=False)
res

In [ ]:
df2.pivot_table(index='model', columns='dataset', values='score_ipo')

In [ ]:
    
cols = [c for c in df2.columns if 'score' in c]
for c in cols:
    df2['correct2'] = df2[c]>0
    df_agg = df2.pivot_table(index='model', columns='dataset', values=c)
    print(c)
    display(df_agg)
    radar_plot(df_agg.T)
    plt.title(c)
    plt.show()


In [ ]:
cols = [c for c in df2.columns if 'score' in c]
for c in cols:
    df2['correct2'] = df2[c]>0
    df_agg = df2.pivot_table(index='model', columns='dataset', values='correct2')
    print(c)
    display(df_agg)
    radar_plot(df_agg.T)
    plt.title(c)
    plt.show()



In [ ]:
# TEST
# TOXIC should be low for most models
# IDMB should be high as it's easy
# random qern should be 50%

In [ ]:
radar_plot(df_agg.T)